<a href="https://colab.research.google.com/github/rgsalcedo/Education/blob/main/AnalisisCuestionario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import files

In [4]:
# Subir el archivo
uploaded = files.upload()

# Obtener el nombre del archivo subido
for filename in uploaded.keys():
    file_path = filename

# Leer el archivo Excel
df = pd.read_excel(file_path)

Saving Respuestas.xlsx to Respuestas.xlsx


In [5]:
# Inicializar el DataFrame para resultados
df_resultado = df.copy()

# Crear una fila vacía como separador usando `pd.NA` en lugar de `None`
separator_row = pd.Series([pd.NA] * df_resultado.shape[1], index=df_resultado.columns)

# Inicializar un diccionario para almacenar los conteos de cada opción para cada pregunta
counts = {'a': [], 'b': [], 'c': [], 'd': []}

# Contar ocurrencias de cada opción para cada pregunta (P1, P2, ..., P15)
for col in df_resultado.columns[1:]:  # Omitir la primera columna que es el ID de los estudiantes
    counts['a'].append((df_resultado[col] == 'a').sum())
    counts['b'].append((df_resultado[col] == 'b').sum())
    counts['c'].append((df_resultado[col] == 'c').sum())
    counts['d'].append((df_resultado[col] == 'd').sum())

# Agregar la fila separadora al DataFrame
df_resultado = pd.concat([df_resultado, pd.DataFrame([separator_row])], ignore_index=True)

# Agregar filas con los conteos de cada opción ('a', 'b', 'c', 'd') y línea en blanco
for option, count_list in counts.items():
    count_row = pd.Series([option] + count_list, index=df_resultado.columns)
    df_resultado = pd.concat([df_resultado, pd.DataFrame([count_row])], ignore_index=True)

# Agregar otra fila separadora debajo de los conteos de opciones
df_resultado = pd.concat([df_resultado, pd.DataFrame([separator_row])], ignore_index=True)

# --- Cálculo adicional: Suma de respuestas correctas por nivel de dificultad ---

# Extraer las respuestas correctas (primera fila, excluyendo la columna de identificación)
respuestas_correctas = df_resultado.iloc[0, 1:]

# Crear un diccionario para almacenar el conteo de respuestas correctas por dificultad
correctas_por_dificultad = {'Baja': 0, 'Media': 0, 'Alta': 0}
preguntas_por_dificultad = {
    'Baja': df_resultado.columns[1:6],  # P1 - P5 (Baja dificultad)
    'Media': df_resultado.columns[6:11],  # P6 - P10 (Media dificultad)
    'Alta': df_resultado.columns[11:16]  # P11 - P15 (Alta dificultad)
}

# Contar respuestas correctas por cada nivel de dificultad
for dificultad, preguntas in preguntas_por_dificultad.items():
    respuestas_estudiantes = df_resultado[1:][preguntas]  # Excluir la fila de respuestas correctas
    correctas_por_dificultad[dificultad] = (respuestas_estudiantes.values == respuestas_correctas[preguntas].values).sum()

# Crear una columna en blanco para separar visualmente
df_resultado[' '] = pd.NA  # Columna en blanco después de `P15`

# Agregar las columnas de suma de respuestas correctas por dificultad al DataFrame
for dificultad, total in correctas_por_dificultad.items():
    df_resultado[f'Respuestas Correctas ({dificultad})'] = [total] + [pd.NA] * (df_resultado.shape[0] - 1)

# --- Cálculo del índice de discriminación para cada pregunta ---

# Replicar respuestas correctas para cada estudiante y restablecer índices en ambos DataFrames
respuestas_correctas_expandido = pd.DataFrame([respuestas_correctas] * (df_resultado.shape[0] - 1),
                                              columns=df_resultado.columns[1:16])
respuestas_correctas_expandido.reset_index(drop=True, inplace=True)
respuestas_estudiantes = df_resultado.iloc[1:, 1:16].reset_index(drop=True)

# Calcular el puntaje total de respuestas correctas para cada estudiante
puntajes_totales = (respuestas_estudiantes == respuestas_correctas_expandido).sum(axis=1)

# Dividir estudiantes en grupos de alto y bajo rendimiento
umbral = int(0.27 * len(puntajes_totales))  # 27% superior e inferior
grupo_superior = puntajes_totales.nlargest(umbral).index
grupo_inferior = puntajes_totales.nsmallest(umbral).index

# Calcular el índice de discriminación por pregunta y colocarlo debajo de cada conjunto de sumas (con una línea en blanco de separación)
for i, col in enumerate(df_resultado.columns[1:16], start=1):  # P1 - P15
    correctos_superior = (df_resultado.loc[grupo_superior, col] == respuestas_correctas[col]).mean()
    correctos_inferior = (df_resultado.loc[grupo_inferior, col] == respuestas_correctas[col]).mean()
    indice_discriminacion = correctos_superior - correctos_inferior

    # Crear la fila de índice de discriminación y agregarla después de los conteos
    discriminacion_row = pd.Series(['Índice de Discriminación'] + [indice_discriminacion] + [pd.NA] * (df_resultado.shape[1] - 2),
                                   index=df_resultado.columns)
    df_resultado = pd.concat([df_resultado, pd.DataFrame([discriminacion_row]), pd.DataFrame([separator_row])],
                             ignore_index=True)
    print(f"Índice de discriminación para {col}: {indice_discriminacion:.2f}")

Índice de discriminación para P1: 0.57
Índice de discriminación para P2: 0.43
Índice de discriminación para P3: 0.29
Índice de discriminación para P4: 0.71
Índice de discriminación para P5: 0.43
Índice de discriminación para P6: 0.71
Índice de discriminación para P7: 0.29
Índice de discriminación para P8: -0.14
Índice de discriminación para P9: 0.57
Índice de discriminación para P10: 0.43
Índice de discriminación para P11: 0.43
Índice de discriminación para P12: 0.00
Índice de discriminación para P13: 0.43
Índice de discriminación para P14: 0.43
Índice de discriminación para P15: 0.43


In [6]:
# --- Cálculo del coeficiente Alfa de Cronbach ---
def calcular_alpha_cronbach(data):
    """
    Calcula el coeficiente alfa de Cronbach para el DataFrame de respuestas
    """
    k = data.shape[1]  # Número de ítems
    varianza_total = data.sum(axis=1).var()  # Varianza de los puntajes totales
    varianza_items = data.var(axis=0).sum()  # Suma de las varianzas de cada ítem

    # Fórmula de Alfa de Cronbach
    alpha = (k / (k - 1)) * (1 - (varianza_items / varianza_total))
    return alpha

# Seleccionar las respuestas de los estudiantes (excluyendo la fila de respuestas correctas y columnas no relacionadas)
respuestas_estudiantes = (df_resultado.iloc[1:, 1:16] == respuestas_correctas.values).astype(int)

# Calcular el alfa de Cronbach
alpha_cronbach = calcular_alpha_cronbach(respuestas_estudiantes)
print(f"Alfa de Cronbach: {alpha_cronbach:.2f}")

# Crear una fila para el alfa de Cronbach y agregarla al final del DataFrame
cronbach_row = pd.Series(['Alfa de Cronbach'] + [alpha_cronbach] + [pd.NA] * (df_resultado.shape[1] - 2),
                         index=df_resultado.columns)
df_resultado = pd.concat([df_resultado, pd.DataFrame([separator_row]), pd.DataFrame([cronbach_row])], ignore_index=True)

# Guardar el DataFrame actualizado en un archivo Excel
df_resultado.to_excel('Resultados_Analisis.xlsx', index=False)

Alfa de Cronbach: 0.96


In [7]:
# --- Análisis de Distractores ---

# Seleccionar las respuestas de los estudiantes (excluyendo la fila de respuestas correctas y columnas adicionales)
respuestas_estudiantes = df_resultado.iloc[1:, 1:16].reset_index(drop=True)

# Asegurar que las respuestas correctas están alineadas y replicadas para cada estudiante
respuestas_correctas_expandido = pd.DataFrame([respuestas_correctas.values] * respuestas_estudiantes.shape[0],
                                              columns=respuestas_estudiantes.columns)

# Calcular el puntaje total de respuestas correctas para cada estudiante (si no se ha hecho ya)
puntajes_totales = (respuestas_estudiantes == respuestas_correctas_expandido).sum(axis=1)

# Dividir estudiantes en grupos de alto y bajo rendimiento (usando los mismos grupos que en el índice de discriminación)
umbral = int(0.27 * len(puntajes_totales))  # 27% superior e inferior
grupo_superior = puntajes_totales.nlargest(umbral).index
grupo_inferior = puntajes_totales.nsmallest(umbral).index

# Análisis de distractores
# Crear una lista para almacenar los resultados
distractor_analysis = []

# Iterar sobre cada pregunta
for col in respuestas_estudiantes.columns:
    correct_answer = respuestas_correctas[col]
    opciones = ['a', 'b', 'c', 'd']
    distractores = [opcion for opcion in opciones if opcion != correct_answer]  # Excluir la respuesta correcta

    for distractor in distractores:
        # Frecuencia de selección del distractor en el grupo de bajo rendimiento
        freq_bajo = (respuestas_estudiantes.loc[grupo_inferior, col] == distractor).mean()
        # Frecuencia de selección del distractor en el grupo de alto rendimiento
        freq_alto = (respuestas_estudiantes.loc[grupo_superior, col] == distractor).mean()

        # Agregar los resultados a la lista
        distractor_analysis.append({
            'Pregunta': col,
            'Distractor': distractor,
            'Frecuencia Bajo Rendimiento': freq_bajo,
            'Frecuencia Alto Rendimiento': freq_alto
        })

        # Imprimir los resultados
        print(f"Distractor '{distractor}' en {col}: Frecuencia bajo rendimiento = {freq_bajo:.2f}, Frecuencia alto rendimiento = {freq_alto:.2f}")

Distractor 'a' en P1: Frecuencia bajo rendimiento = 0.00, Frecuencia alto rendimiento = 0.00
Distractor 'c' en P1: Frecuencia bajo rendimiento = 0.00, Frecuencia alto rendimiento = 0.00
Distractor 'd' en P1: Frecuencia bajo rendimiento = 0.00, Frecuencia alto rendimiento = 0.07
Distractor 'a' en P2: Frecuencia bajo rendimiento = 0.00, Frecuencia alto rendimiento = 0.13
Distractor 'b' en P2: Frecuencia bajo rendimiento = 0.00, Frecuencia alto rendimiento = 0.00
Distractor 'c' en P2: Frecuencia bajo rendimiento = 0.00, Frecuencia alto rendimiento = 0.00
Distractor 'a' en P3: Frecuencia bajo rendimiento = 0.00, Frecuencia alto rendimiento = 0.13
Distractor 'b' en P3: Frecuencia bajo rendimiento = 0.00, Frecuencia alto rendimiento = 0.00
Distractor 'd' en P3: Frecuencia bajo rendimiento = 0.00, Frecuencia alto rendimiento = 0.00
Distractor 'a' en P4: Frecuencia bajo rendimiento = 0.00, Frecuencia alto rendimiento = 0.00
Distractor 'b' en P4: Frecuencia bajo rendimiento = 0.00, Frecuencia a

In [9]:
# --- Cálculo del Análisis de Dificultad de Ítems ---

# Extraer las respuestas correctas de la primera fila (excluyendo la columna de identificación)
# Asegurarse de usar el nombre de la columna si no coincide con el índice esperado
respuestas_correctas = df_resultado.iloc[0, 1:16].reset_index(drop=True)

# Crear una lista para almacenar los resultados del análisis de dificultad
dificultad_items = []

# Iterar sobre cada columna (pregunta) para calcular la dificultad
for idx, col in enumerate(df_resultado.columns[1:16], start=1):  # P1 - P15
    # Verificar que estamos trabajando con la columna correcta y que no contiene índices no deseados
    try:
        # Contar el número de estudiantes que respondieron correctamente la pregunta
        num_correctas = (df_resultado.iloc[1:, idx] == respuestas_correctas[idx - 1]).sum()
        # Calcular el total de estudiantes que respondieron la pregunta
        num_total = df_resultado.iloc[1:, idx].notna().sum()

        # Calcular la dificultad del ítem (proporción de respuestas correctas)
        dificultad = num_correctas / num_total

        # Clasificar el nivel de dificultad según el valor de dificultad calculado
        if dificultad >= 0.7:
            nivel_dificultad = 'Baja'
        elif 0.3 <= dificultad < 0.7:
            nivel_dificultad = 'Media'
        else:
            nivel_dificultad = 'Alta'

        # Agregar los resultados a la lista
        dificultad_items.append({
            'Pregunta': col,
            'Dificultad': dificultad,
            'Nivel de Dificultad': nivel_dificultad
        })

        # Imprimir el resultado para verificar
        print(f"Pregunta {col}: Dificultad = {dificultad:.2f}, Nivel de Dificultad = {nivel_dificultad}")

    except Exception as e:
        print(f"Error procesando la pregunta {col}: {e}")

# Convertir los resultados en un DataFrame
df_dificultad = pd.DataFrame(dificultad_items)

Pregunta P1: Dificultad = 0.45, Nivel de Dificultad = Media
Pregunta P2: Dificultad = 0.67, Nivel de Dificultad = Media
Pregunta P3: Dificultad = 0.67, Nivel de Dificultad = Media
Pregunta P4: Dificultad = 0.75, Nivel de Dificultad = Baja
Pregunta P5: Dificultad = 0.42, Nivel de Dificultad = Media
Pregunta P6: Dificultad = 0.79, Nivel de Dificultad = Baja
Pregunta P7: Dificultad = 0.29, Nivel de Dificultad = Alta
Pregunta P8: Dificultad = 0.33, Nivel de Dificultad = Media
Pregunta P9: Dificultad = 0.62, Nivel de Dificultad = Media
Pregunta P10: Dificultad = 0.46, Nivel de Dificultad = Media
Pregunta P11: Dificultad = 0.58, Nivel de Dificultad = Media
Pregunta P12: Dificultad = 0.17, Nivel de Dificultad = Alta
Pregunta P13: Dificultad = 0.58, Nivel de Dificultad = Media
Pregunta P14: Dificultad = 0.62, Nivel de Dificultad = Media
Pregunta P15: Dificultad = 0.58, Nivel de Dificultad = Media


In [13]:
#-------- Guardar y descargar el archivo resultante ---------------

# Guardar el DataFrame actualizado en un archivo Excel
df_resultado.to_excel('Resultados_Analisis.xlsx', index=False)

# Descargar el archivo en tu computadora (solo para Google Colab)
files.download('Resultados_Analisis.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>